Accessing Data from Google Drive

In [ ]:
import os,sys
from google.colab import drive
drive.mount('/content/gdrive')
nb_path = "/content/notebooks"
os.symlink("/content/gdrive/MY Drive/Colab Note Notebooks",nb_path)
sys.path.insert(0,nb_path)

Install Required Packages

In [ ]:
!pip install --target=$nb_path  soundfile

  Using cached https://files.pythonhosted.org/packages/eb/f2/3cbbbf3b96fb9fa91582c438b574cff3f45b29c772f94c400e2c99ef5db9/SoundFile-0.10.3.post1-py2.py3-none-any.whl


Import Required Packages

In [47]:
import librosa,soundfile
import glob,pickle
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

Manipulate data for model

In [48]:
def Extract_Feature(file_name,mfcc,chroma,mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate = sound_file.samplerate
    result = np.array([])
    if mfcc:
      mfcc = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=40).T,axis=0)
      result = np.hstack((result,mfcc))
    if chroma:
      stft = np.abs(librosa.stft(X))
      chroma = np.mean(librosa.feature.chroma_stft(sr=sample_rate,S=stft).T,axis=0)
      result = np.hstack((result,chroma))
  if mel:
    mel = np.mean(librosa.feature.melspectrogram(X,sr=sample_rate).T,axis=0)
    result = np.hstack((result,mel))
    #print("$$",result)
  return result

In [49]:
emotions = {'01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'}
observed_emotions = ['calm','happy','fearful','disgust']

In [50]:
def Load_Data(test_size=0.2):
  x,y = [],[]
  for file in glob.glob('/content/gdrive/My Drive/Dataset/speech-emotion-recognition-ravdess-data/Actor_*/*.wav'):
    file_name = os.path.basename(file)
    emotion = emotions[file_name.split('-')[2]]
    if emotion not in observed_emotions:
      continue
    feature = Extract_Feature(file, mfcc=True, chroma=True, mel=True)
    x.append(feature)
    y.append(emotion)
  return train_test_split(np.array(x),y,test_size=test_size,random_state=9)

Splitting Dataset into Training and Testing

In [51]:
X_train,X_test,y_train,y_test = Load_Data(test_size=0.2)

In [52]:
print(X_train.shape,X_test.shape)

(618, 180) (155, 180)


In [53]:
print(X_train[0],y_train[0])

[-4.81413306e+02  4.03593706e+01 -1.13141032e+01 -8.03662432e+00
 -8.66522752e+00 -4.50140331e+00 -2.16066759e+01 -1.62738401e+01
 -1.06776759e+01 -2.36751501e+00 -6.76163545e+00 -6.64711897e+00
 -8.24356827e+00 -4.89691398e+00 -7.86083205e+00 -2.67248495e+00
 -4.93339878e+00 -4.68338945e+00 -8.48190321e+00 -2.66674083e+00
 -2.19513791e+00  1.78093641e+00  1.14951632e+00  1.57652972e-01
  2.09153066e+00  3.48092997e+00  1.74600478e+00  2.76986271e+00
  2.94236244e+00  2.06425971e+00  1.85070871e+00  2.95802651e+00
  7.58656380e-01  1.53913808e+00 -9.79123146e-01 -1.24428534e+00
 -1.82763570e+00 -3.20952285e-02 -7.70709269e-01  1.21527977e+00
  5.01902008e-01  5.34202836e-01  5.79923974e-01  6.38008660e-01
  6.73507896e-01  6.35513030e-01  6.38416730e-01  7.02088253e-01
  7.19661309e-01  6.94322258e-01  6.61440118e-01  6.00765089e-01
  6.68950796e-05  3.55241215e-05  1.05166130e-04  2.46903020e-04
  4.19248289e-04  3.84587902e-03  3.63136688e-02  9.05588845e-02
  2.87115120e-01  5.32530

Creating Model

In [54]:
model = MLPClassifier(alpha=0.01,learning_rate='adaptive',max_iter=500,batch_size=256,epsilon=1e-8,hidden_layer_sizes=(300,))

Evaluating Model

In [56]:
model.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [57]:
y_pred = model.predict(X_test)

In [58]:
accuracy = accuracy_score(y_pred,y_test)
print(f"Accuracy : {round(accuracy*100,2)}%")

Accuracy : 73.55%
